In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator


slim_orca = load_dataset("Open-Orca/SlimOrca")
# slim_orca = slim_orca.train_test_split(test_size=0.2)

texts = [conversation["text"] for conversation in slim_orca["train"]]


In [ ]:
print(slim_orca["test"])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
tokenized_texts = [tokenizer(text) for text in texts]


In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("google/flan-t5-large")

In [ ]:
data_collator = DefaultDataCollator()

In [ ]:
training_args = TrainingArguments(
    output_dir="slimorca-ft5l",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=slim_orca["train"],
    eval_dataset=slim_orca["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()